# PROBLEM 2

## Prebacivanje epuba u txt te korištenje regexa za čišćenje

In [55]:
import ebooklib
import re
from ebooklib import epub
def epub2thtml(epub_path):
    book = epub.read_epub(epub_path)
    chapters = []
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            chapters.append(item.get_content())
    return chapters

from bs4 import BeautifulSoup
blacklist = [   '[document]',   'noscript', 'header',   'html', 'meta', 'head','input', 'script', 'xa0', 'n' ]
# there may be more elements you don't want, such as "style", etc.

def chap2text(chap):
    output = ''
    soup = BeautifulSoup(chap, 'html.parser')
    text = soup.find_all(text=True)
    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)
    return output

def thtml2ttext(thtml):
    Output = []
    for html in thtml:
        text =  chap2text(html)
        Output.append(text)
    return Output

def epub2text(epub_path):
    chapters = epub2thtml(epub_path)
    ttext = thtml2ttext(chapters)
    return ttext

def listToString(s):
    str1 = ""
    for elem in s:
        str1 += elem
    return str1

out=epub2text('brlicmazuranic_basne.epub')
outString = listToString(out)
plainText=re.sub(r'[~%&\\<>?#]+(r|n|xa0)',"",outString)

In [56]:
basneTxt = open("basne","w")
basneTxt.write(str(plainText))

28502

### basne.txt Spreman za  korištenje

In [2]:
import io
with io.open('basne', encoding='utf-8') as fin:
    text=fin.read()

In [3]:
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk import word_tokenize, sent_tokenize
from nltk.lm.preprocessing import padded_everygram_pipeline


In [10]:
spl = int(80*len(text)/100)
train = text[:spl]
test = text[spl:]
tokenized_text = [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(train)]

In [12]:
n=3 #trigram
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)

## Treniranje `N-GRAM` modela

In [13]:
from nltk.lm import MLE
model = MLE(n)

In [14]:
model.fit(train_data, padded_sents)

## Generiranje jezika korištenjem `N-GRAM` jezičnog modela

In [15]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(model, num_words, random_seed=42):
    content = []
    for token in model.generate(num_words, random_seed = random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

In [16]:
generate_sent(model,20,random_seed=11)

'dobrotvor soko ptičicu hrani.'

In [17]:
generate_sent(model,10, random_seed=42)

'mu dadem dobar zalogaj!'

## Spremanje modela u serijalizator objekta `pickle`

In [19]:
import dill as pickle
with (open('ngram_model.pkl', 'wb')) as fout:
    pickle.dump(model,fout)

Učitavanje modela:

In [20]:
with (open('ngram_model.pkl', 'rb')) as fin:
    model_loaded = pickle.load(fin)

Funkcija koja generira 10 random rečenica

In [21]:
def generate10(model):
    import random
    x = 0
    while(x<10):
        a = random.randrange(1,20)
        b = random.randrange(1,100)
        sent = generate_sent(model,a,random_seed=b)
        print(sent)
        x+=1
    return

In [24]:
generate10(model_loaded)

šegrt hlapić i danas je među najomiljenijim dječjim knjigama, oba
hladom mališ se pati, krilca ga gola ne mogu dić.
pas započme govor lijep
biva, ele, soko ovog
će, al na svoju pustu nesreću ili sreću nikuda iz svoje kaljuže ne kreću
] titra, a veliko zvijere!
hladom mališ se pati, krilca ga

u starogrčkoj arhitekturi sa karakterističnim užlijebljenim stupovima (drugi stilovi
a što mi ga radiš?
